In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import sqlalchemy as db
from snowflake.sqlalchemy import URL

In [2]:
snowflake_credentials = pd.read_excel('/Users/prai/Desktop/Developement/snowflake_credentials.xlsx')
global user
global password
global account
global warehouse
global schema
user = str(snowflake_credentials.iloc[0,1])
password = str(snowflake_credentials.iloc[1,1])
account = str(snowflake_credentials.iloc[2,1])
warehouse = str(snowflake_credentials.iloc[3,1])
schema = str(snowflake_credentials.iloc[4,1])

global sfengine
sfengine = db.create_engine(URL(
    user = user
    ,password = password
    ,account = account
    ,warehouse = warehouse
    ,database = "TRANSFORM"
    ,schema = schema
))

In [18]:
input_global_id ='e2f7b101-60ed-448c-82a7-a6d200b7f02d'
query="select pt.name as Pricing_Model,p.startingprice as Cost, pf.name as Payment_Frequency "+\
    "from replicon.upc.product p "+\
    "join replicon.upc.pricingmodeltype pt "+\
    "on pt.pricingmodeltypeid = p.pricingmodeltypeid "+\
    "join replicon.upc.PAYMENTFREQUENCYTYPE pf "+\
    "on pf.PAYMENTFREQUENCYTYPEID = p.PAYMENTFREQUENCYTYPEID "+\
    "where lower(productintegrationid) = '"+str(input_global_id)+"'"
cost_payment = pd.read_sql_query(query, sfengine)
cost_payment[['Pricing_Model','Cost','Payment_Frequency']]=cost_payment[['pricing_model','cost','payment_frequency']]
cost_payment=cost_payment.drop(['pricing_model','cost','payment_frequency'],axis=1)
# cost_payment=cost_payment.transpose()
cost_payment['Product'] = "*Product*"
cost_payment=cost_payment[['Product','Pricing_Model','Cost','Payment_Frequency']]
cost_payment.to_csv('/Users/prai/Desktop/cost_payment_line1.csv')

In [39]:
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
cost_payment=pd.read_csv('/Users/prai/Desktop/cost_payment_line1.csv')

cost_payment = cost_payment.loc[:, ~cost_payment.columns.str.contains('^Unnamed')]
plt.savefig('/Users/prai/Desktop/table_plotly.png', bbox_inches='tight',transparent=False)
# fig =  ff.create_table(cost_payment)
# fig.update_layout(
#     autosize=False,
#     width=500,
#     height=100,
# )
# fig.write_image("/Users/prai/Desktop/table_plotly.png", scale=1)
fig.show()

<Figure size 432x288 with 0 Axes>

In [9]:
import docx
from docx.shared import Inches
# Create an instance of a word document
doc = docx.Document()
  
# Add a Title to the document
doc.add_heading('Gartner Sample Data', 3)
doc.add_picture('/Users/prai/Desktop/images.png')
doc.add_picture('/Users/prai/Desktop/table_plotly.png', width=Inches(6))
# Table data in a form of list
data = (
    ("Author name", "Author = The name of the author who wrote the piece. Also include the name that should appear in the byline, if it differs."),
    ("H1", '"H1 = Title at the top of the content; the headline for the piece"'),
    ("Title Tag (< 60 characters)", "Title Tags = “Title” that appears in the SERPs - Often the same as the H1, unless the H1 is too many characters - Under 60 characters (preview it here)- Target keywords should be as close to the front as possible- Should be descriptive of the content, but also unique to stand out in the SERPs (Be sure to Google your targeted keyword and see how you can differentiate from what already there)"),
    ("Meta Description (150-170 characters)","Meta Descriptions = Description under the TT that appears in the SERPs - 150 to 170 characters - Should be detailed, relevant and eye-catching - Include keywords (this has no direct SEO impact, but they do appear bolded to catch a searcher's attention) - Use action-oriented language (e.g., Learn how... Discover how...Find out....) Resource: HubSpot has a great blog on the subject of writing meta descriptions: http://blog.hubspot.com/marketing/how-to-write-meta-description-ht."),
    ("URL","URLs  Match the URL as closely as possible to your highest volume keyword - Use hyphens to separate words (e.g., hipaa-violations-in-the-news)  Avoid using numbers, especially years or the number of products in a list piece unless approved by SEO  Always use all lowercase"),
    ("Keyword/volume"," "),
    ("Market or Blog Category:","Choose from column C in this list"),
    ("[Capterra Only] Subheadline","A secondary headline that expands on the title and compels the audience to keep reading. Appears under the H1 and above the header image in Capterra blog content. Appears above the header image Example."))
  
# Creating a table object
table = doc.add_table(rows=1, cols=2)
  
# Adding heading in the 1st row of the table
row = table.rows[0].cells
row[0].text = 'Column 1'
row[1].text = 'Column 2'
  
# Adding data from the list to the table
for id, name in data:
  
    # Adding a row and then adding data in it.
    row = table.add_row().cells
    # Converting id to string as table can only take string input
    row[0].text = str(id)
    row[1].text = name
table.name = 'MS Gothic'
table.size = docx.shared.Pt(15)
table.style = 'TableGrid'
font = table.style.font
font.bold= True

doc.add_page_break()
doc.save('/Users/prai/Desktop/Table_template.docx')

/Users/prai/opt/anaconda3/lib/python3.8/site-packages/docx/styles/styles.py:139: UserWarning:

style lookup by style_id is deprecated. Use style name as key instead.

